# Duckietown and ROS

In this notebook we will use what we have learned about ROS in the previous notebooks to start to control the duckiebot, either in simulation or on real hardware. 

## See what the Duckiebot Sees

As before, launch the noVNC browser by running following command in the terminal **of your laptop** (not the terminal in VSCode!) except this time with the `--sim` argument:

    dts code workbench --sim

Note 1: You will need to stop the previous with CTRL-C and then execute this with the `--sim` option.
Note 2: This may take some time to run the first time because it is downloaded the docker image of the Duckietown simulation environment. 

Once you open the noVNC browser, as before, you should see the same icons on the left hand side. Double-click on the one that says `RQT Image ...`, clicking on this opens the [rqt_image_view](https://wiki.ros.org/rqt_image_view) ROS utility.

In the top left scroll down bar of the `rqt_image_view` window you should see only one option: `/agent/camera_node/image/compressed` if you select it you will see the output from the camera on the Duckiebot.

![rqt_image_view](../assets/rqt_image_view.png)

## Try out the Joystick

You can also open the joystick by double clicking on the `Joystick` icon. You can try clicking on the directions or using the arrow keys on your keyboard. You should see the corresponding direction on the joystick turn green, but you won't see robot moving in the simulation (assuming you still have the `rqt_image_view` window open from the previous step).

However, the joystick **is** doing something. To see what it is doing, open up a terminal by clicking on the `LXTerminal` icon as before. Execute the following command on the terminal:

    rostopic echo /agent/joy

The click back to the joystick window and start clicking on the directions and look closely at the output that is produced in the terminal. In particular, look at the `axes` part of the data that is produced. You should see one of the values changing (the position may change and the value may be either a `+1` or a `-1` depending on which direction you push).

![joy_echo](../assets/joy_echo.png)

Finally, do the following in the terminal

    rostopic list

to see all of the topics:

```
/agent/camera_node/camera_info
/agent/camera_node/image/compressed
/agent/coordinator_node/intersection_go
/agent/episode_start
/agent/joy
/agent/led_emitter_node/led_pattern
/agent/left_wheel_encoder_node/tick
/agent/right_wheel_encoder_node/tick
/agent/wheels_driver_node/emergency_stop
/agent/wheels_driver_node/wheels_cmd
/rosout
/rosout_agg
```

we can see the topic `/agent/joy` and we know at least how that data is being published. This message is of type [Joy.msg](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html) (this is a standard datatype provided by ROS). The topic that we need to publish messages on to make the robot actually move is `/agent/wheels_driver_node/wheels_cmd`. This message is of type [WheelsCmdStamped.msg](https://github.com/duckietown/dt-ros-commons/blob/daffy/packages/duckietown_msgs/msg/WheelsCmdStamped.msg) (this is a message that we defined and is available to us because it is defined in an upstream Docker image).

Our objective for the rest of this exercise will be create a ROS node to take the messages that are being published on the `/agent/joy` topic and use them to publish something to the `/agent/wheels_driver_node/wheels_cmd` topic so that the robot moves and we can control it with the joystick.  

## Create a New Package

Use the menu on the left of this notebook to open a new terminal (three lines -> Terminal -> New Terminal). In that terminal, navigate to the `packages` directory:

    cd packages

Once there we can proceed to create our package. Let's call it `dt-joystick-demo`. You can create it by executing the following code in your terminal

    catkin_create_pkg dt-joystick-demo std_msgs sensor_msgs duckietown_msgs rospy

You should see the output:

```
Created file dt-joystick-demo/package.xml
Created file dt-joystick-demo/CMakeLists.txt
Successfully created files in /code/lx-ros-basics/packages/dt-joystick-demo. Please adjust the values in package.xml.
```

let's enter into the `dt-joystick-demo` directory:

    cd dt-joystick-demo

then make a `src` dirctory:

    mkdir src

now enter that `src` directory:

    cd src

First we need to create the file `__init__.py`

    touch __init__.py

and then create a python file:

    touch dt-joystick-demo-node.py

and finally we need to make that file execuatable:

    chmod +x dt-joystick-demo-node.py

You should now be able to navigate to that file using the file directory on the left. Click on the file to open it (you may want to split screen using the button at the top right ). 

## Writing our ROS Node

We need to start with some basics. First off we need the following at the top of the file:

```
#!/usr/bin/env python3
```

This tells the python interpreter that this is a python file. 

Next we will need to import some packages, these will by `rospy` and then the types of *messages* we will be using later.

```
import rospy
from sensor_msgs.msg import Joy
from duckietown_msgs.msg import WheelsCmdStamped
```

Now let's define our node. We can start with 

```
def dt_joystick_node():
```

(and then everything after this will need to be indented)